In [1]:
from fastapi import FastAPI
from starlette.exceptions import HTTPException
from fastapi.middleware.cors import CORSMiddleware
from starlette.status import HTTP_422_UNPROCESSABLE_ENTITY

#from .api.api_v1.api import router as api_router
from core.config import ALLOWED_HOSTS, PROJECT_NAME, API_PORT, API_V1_STR
from core.errors import http_422_error_handler, http_error_handler
from db.mongodb_connect import close_mongo_connection, connect_to_mongo
from db.mongodb import AsyncIOMotorClient, get_database
import asyncio

In [2]:
app = FastAPI(title=PROJECT_NAME)

if not ALLOWED_HOSTS:
    ALLOWED_HOSTS = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=ALLOWED_HOSTS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


app.add_event_handler("startup", connect_to_mongo)
app.add_event_handler("shutdown", close_mongo_connection)


app.add_exception_handler(HTTPException, http_error_handler)
app.add_exception_handler(HTTP_422_UNPROCESSABLE_ENTITY, http_422_error_handler)

In [3]:

from api.api_v1.api import router as api_v1_router
from api.api_webchat.api import router as api_webchat_router
from api.api_developer.api import router as api_developer_router

#app.include_router(api_v1_router, prefix=API_V1_STR, tags=["telegram"])
app.include_router(api_webchat_router, prefix="/api/webchat", tags=["webchat"])
app.include_router(api_developer_router, prefix="/api/developer", tags=["developer"])

In [4]:
from fastapi.responses import HTMLResponse, FileResponse
import os

@app.get("/")
def home():
    with open(f"{static_file_path}/index.html") as f:
        html = "".join(f.readlines())
    return HTMLResponse(content=html, status_code= 200)

@app.get("/{whatever:path}")
async def get_static_files_or_404(whatever):
    # try open file for path
    file_path = os.path.join(static_file_path,whatever)
    if os.path.isfile(file_path):
        return FileResponse(file_path)
    return FileResponse(f"{static_file_path}/index.html")

In [5]:
@app.get("/index2")
def fun():
    with open("./static/index2.html") as f:
        html = "".join(f.readlines())
    return HTMLResponse(content=html, status_code= 200)

In [6]:
static_file_path = "../front-end/dist"
from fastapi.staticfiles import StaticFiles
app.mount("/", StaticFiles(directory=static_file_path), name="static")


In [ ]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=API_PORT)

INFO:     Started server process [29607]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:13525 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://0.0.0.0:13525 (Press CTRL+C to quit)


INFO:     36.228.93.139:0 - "GET / HTTP/1.1" 200 OK
../front-end/dist/css/app.77c6ebc6.css
../front-end/dist/css/chunk-vendors.188e04d6.css
INFO:     36.228.93.139:0 - "GET /css/chunk-vendors.188e04d6.css HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "GET /css/app.77c6ebc6.css HTTP/1.1" 200 OK
../front-end/dist/js/chunk-vendors.4112a27b.js
../front-end/dist/js/app.4206fd8c.js
INFO:     36.228.93.139:0 - "GET /js/chunk-vendors.4112a27b.js HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "GET /js/app.4206fd8c.js HTTP/1.1" 200 OK
../front-end/dist/css/chunk-35b9d5ef.b2009534.css
../front-end/dist/js/chunk-35b9d5ef.053ebd83.js
INFO:     36.228.93.139:0 - "GET /css/chunk-35b9d5ef.b2009534.css HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "GET /js/chunk-35b9d5ef.053ebd83.js HTTP/1.1" 200 OK
../front-end/dist/img/artificial-intelligence-robots-cyborg-with-infinity-symbol.jpg
../front-end/dist/img/icons/favicon-32x32.png
INFO:     36.228.93.139:0 - "GET /img/artificial-intelligence-robots-cyborg-wit

INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "GET /online/chatroom/ HTTP/1.1" 200 OK
../front-end/dist/css/app.77c6ebc6.css
INFO:     36.228.93.139:0 - "GET /css/app.77c6ebc6.css HTTP/1.1" 200 OK
../front-end/dist/css/chunk-vendors.188e04d6.css
INFO:     36.228.93.139:0 - "GET /css/chunk-vendors.188e04d6.css HTTP/1.1" 200 OK
../front-end/dist/js/app.4206fd8c.js
INFO:     36.228.93.139:0 - "GET /js/app.4206fd8c.js HTTP/1.1" 200 OK
../front-end/dist/js/chunk-vendors.4112a27b.js
INFO:     36.228.93.139:0 - "GET /js/chunk-vendors.4112a27b.js HTTP/1.1" 200 OK
../front-end/dist/css/chunk-35b9d5ef.b2009534.css
INFO:     36.228.93.139:0 - "GET /css/chunk-35b9d5ef.b2009534.css HTTP/1.1" 200 OK
../front-end/dist/js/chunk-35b9d5ef.053ebd83.js
INFO:     36.228.93.139:0 - "

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 396, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/middlew

INFO:     36.228.93.139:0 - "GET /favicon.ico HTTP/1.1" 500 Internal Server Error


...

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 396, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/middlew

INFO:     36.228.93.139:0 - "GET /favicon.ico HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/uvicorn/protocols/http/h11_impl.py", line 396, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/home/eason/anaconda3/envs/ser/lib/python3.8/site-packages/starlette/middlew

INFO:     36.228.93.139:0 - "GET /online/chatroom/ HTTP/1.1" 200 OK
../front-end/dist/js/app.ba565489.js
INFO:     36.228.93.139:0 - "GET /js/app.ba565489.js HTTP/1.1" 200 OK
../front-end/dist/js/chunk-vendors.4112a27b.js
INFO:     36.228.93.139:0 - "GET /js/chunk-vendors.4112a27b.js HTTP/1.1" 200 OK
../front-end/dist/css/chunk-vendors.188e04d6.css
INFO:     36.228.93.139:0 - "GET /css/chunk-vendors.188e04d6.css HTTP/1.1" 200 OK
../front-end/dist/css/app.77c6ebc6.css
INFO:     36.228.93.139:0 - "GET /css/app.77c6ebc6.css HTTP/1.1" 200 OK
../front-end/dist/css/chunk-35b9d5ef.b2009534.css
../front-end/dist/js/chunk-35b9d5ef.053ebd83.js
INFO:     36.228.93.139:0 - "GET /css/chunk-35b9d5ef.b2009534.css HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "GET /js/chunk-35b9d5ef.053ebd83.js HTTP/1.1" 200 OK
../front-end/dist/js/app.ba565489.js.map
INFO:     36.228.93.139:0 - "GET /js/app.ba565489.js.map HTTP/1.1" 200 OK
../front-end/dist/fonts/nucleo-outline.8ebec31f.woff2
INFO:     36.228.93.139:0

INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK
INFO:     36.228.93.139:0 - "POST /api/webchat/generate_response HTTP/1.1" 200 OK


In [ ]:
print("讚美主")